In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from BSDE import *
import warnings 
import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show, output_file
import time
warnings.filterwarnings("ignore")

In [11]:
T = 0.5
K = 100.
r = 0.04
R = 0.06
p = 1
M = np.eye(p)
S_init = 100.
mu = 0.06
sigma = 0.2
Q = 0

In [17]:
N = 10000
m = 6
RF_n_trees = 200
RF_max_leaf_nodes = 100
M_run = 20
n_picard = 3

In [16]:
test = BsdeHD(T, K, M, mu, Q, sigma, S_init, r, R)
test.get_price(N, m, option_type='call', option_payoff='geometric',
               oType='European',n_picard=n_picard, RF_n_estimators = 200, RF_max_leaf_nodes=300)

7.2756003467624302

In [22]:
range_leaf = len(list(range(50, 10000, 50)))
a = np.zeros([range_leaf, 10])
timing = np.zeros_like(a)
for i, leaf in enumerate(list(range(50, 10000, 50))): 
    for j in range(10): 
        test = BsdeHD(T, K, M, mu, Q, sigma, S_init, r, R)
        a[i, j] = round(test.get_price(N, m, option_type='call', option_payoff='geometric',
                                       oType='European',n_picard=n_picard, RF_n_estimators = 200, RF_max_leaf_nodes=leaf), 3)      
    print (a[i])   

[ 7.235  7.328  7.328  7.347  7.209  7.217  7.209  7.231  7.167  7.216]


KeyboardInterrupt: 

In [10]:
a

array([ 3.52532364,  3.51318395,  3.47813026,  3.52643139,  3.46771921,
        3.55057315,  3.47298629,  3.41433999,  3.51075648,  3.51146446,
        3.52424889,  3.50691624,  3.48597705,  3.43988207,  3.48719892,
        3.4937815 ,  3.52544215,  3.47189406,  3.47695785,  0.        ])

In [13]:
import pandas as pd

In [14]:
param = ['q', 'mu', 'R ', 'r ','sigma', 'T', 'K', 'S0']
values = [Q, mu, R, r, sigma, T, K, S0]
dataset = list(zip(param,values))
df_param = pd.DataFrame(data = dataset, columns=['Parameters', 'values'])

In [15]:
sparam = ['N_particles', 'm_discretization', 'N_run', 'n_picard','Number of trees', 'Max Leaf nodes']
svalues = [N, m, M_run, n_picard, RF_n_trees, RF_max_leaf_nodes]
sp_dataset = list(zip(sparam,svalues))
df_sparam = pd.DataFrame(sp_dataset, columns=['Specific Parameters', 'values'])
df_sparam

,Specific Parameters,values
0,N_particles,10000
1,m_discretization,12
2,N_run,20
3,n_picard,10
4,Number of trees,200
5,Max Leaf nodes,100


In [16]:
run_id = ["run {}".format(i) for i in range(1, M_run+1)]
df = pd.DataFrame(list(zip(run_id, a, timing)), columns = ['run','values', 'time'])

In [17]:
min_a = round(df['values'].min(), 4)
max_a = round(df['values'].max(), 4)
mean_a = round(df['values'].mean(), 4)
std_a = round(df['values'].std(), 4)
left = round(mean_a - 1.96 * std_a / M_run, 4)
right = round(mean_a + 1.96 * std_a / M_run, 4)

In [18]:
stat_param = ['mean', 'std', '95% confidence interval', 'min', 'max']
stat_values = [mean_a, std_a, [left, right], min_a, max_a]
stat_data = list(zip(stat_param, stat_values))
stat_df = pd.DataFrame(stat_data, columns = ['stat parameter', 'values'])
stat_df

,stat parameter,values
0,mean,7.2331
1,std,0.0979
2,95% confidence interval,"[7.2235, 7.2427]"
3,min,7.0684
4,max,7.4308


In [19]:
df_join = pd.concat([df_param, df_sparam, df, stat_df], axis = 0)
df_join.to_csv('output_rf.csv')